In [3]:
"""
@Description :   
@Author      :   Xubo Luo 
@Time        :   2024/07/10 20:09:58
"""

from lightglue.modules import LightGlue, SuperPoint, DISK
from lightglue.modules.utils import load_image, rbd
from lightglue.modules import viz2d
import torch
import cv2
from xfeat.modules.xfeat import XFeat

ModuleNotFoundError: No module named 'model'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'
matcher = LightGlue(features="superpoint").eval().to(device)
xfeat = XFeat()

In [ ]:
image0 = cv2.imread("../assets/ref.png")
image1 = cv2.imread("../assets/tgt.png")

# Extract keypoints and descriptors
image0 = XFeat.parse_input(image0)
image1 = XFeat.parse_input(image1)

feat0 = XFeat.detectAndCompute(image0, top_k = 2048)[0]
feat1 = XFeat.detectAndCompute(image1, top_k = 2048)[0]

kpts0, desc0 = feat0["keypoints"], feat0["descriptors"]
kpts1, desc1 = feat1["keypoints"], feat1["descriptors"]

kpts0 = kpts0.unsqueeze(0)
kpts1 = kpts1.unsqueeze(0)

In [ ]:
matches01 = matcher({"image0":feat0, "image1":feat1})

feat0, feat1, matches01 = [
    rbd(x) for x in (feat0, feat1, matches01)
]   # remove batch dimension

kpts0, kpts1, matches = feat0["keypoints"], feat1["keypoints"], matches01["matches"]
m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]

In [ ]:
axes = viz2d.plot_images([image0, image1])
viz2d.plot_matches(m_kpts0, m_kpts1, color="lime", lw=0.2)
viz2d.add_text(0, f'Stop after {matches01["stop"]} layers', fs=20)